<a href="https://colab.research.google.com/github/amiranju/planets/blob/master/Copy_of_20_timeseries_inspector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/20_timeseries_inspector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap

In [ ]:
geemap.show_youtube("0CZ7Aj8hCyo")

## Update the geemap package

If you run into errors with this notebook, please uncomment the line below to update the [geemap](https://github.com/gee-community/geemap#installation) package to the latest version from GitHub.
Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [ ]:
# geemap.update_package()

## NAIP: National Agriculture Imagery Program

The National Agriculture Imagery Program (NAIP) acquires aerial imagery during the agricultural growing seasons in the continental U.S.

NAIP projects are contracted each year based upon available funding and the FSA imagery acquisition cycle. Beginning in 2003, NAIP was acquired on a 5-year cycle. 2008 was a transition year, and a three-year cycle began in 2009.

NAIP imagery is acquired at a **one-meter** ground sample distance (GSD) with a horizontal accuracy that matches within six meters of photo-identifiable ground control points, which are used during image inspection.

Older images were collected using 3 bands (Red, Green, and Blue: RGB), but newer imagery is usually collected with an additional near-infrared band (RGBN).

More information about NAIP imagery can be found on [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ).

## Create annual composite of NAIP imagery

Select 4-band (RGBN) NAIP imagery.

In [ ]:
Map = geemap.Map()

In [ ]:
naip_ts = geemap.naip_timeseries(start_year=2009, end_year=2018)

Create a list of layer names to be shown under the dropdown list.

In [ ]:
layer_names = ["NAIP " + str(year) for year in range(2009, 2019)]
print(layer_names)

Set visualization parameters.

In [ ]:
naip_vis = {"bands": ["N", "R", "G"]}

Create a split-panel map for visualizing NAIP imagery

In [ ]:
Map = geemap.Map()
Map.ts_inspector(
    left_ts=naip_ts,
    right_ts=naip_ts,
    left_names=layer_names,
    right_names=layer_names,
    left_vis=naip_vis,
    right_vis=naip_vis,
)
Map

## Create annual composite of Landsat imagery

Use the drawing tools to create an ROI

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

In [ ]:
region = Map.draw_last_feature

In [ ]:
if region is not None:
    roi = region.geometry()
else:
    roi = ee.Geometry.Polygon(
        [
            [
                [-115.897448, 35.640766],
                [-115.897448, 36.603608],
                [-113.784915, 36.603608],
                [-113.784915, 35.640766],
                [-115.897448, 35.640766],
            ]
        ],
        None,
        False,
    )

In [ ]:
print(roi.getInfo())

In [ ]:
landsat_ts = geemap.landsat_timeseries(
    roi=roi, start_year=1984, end_year=2019, start_date="01-01", end_date="12-31"
)

In [ ]:
layer_names = ["Landsat " + str(year) for year in range(1984, 2020)]
print(layer_names)

In [ ]:
landsat_vis = {
    "min": 0,
    "max": 0.3,
    "gamma": [1, 1, 1],
    "bands": ["NIR", "Red", "Green"],
}

In [ ]:
Map = geemap.Map()
Map.ts_inspector(
    left_ts=landsat_ts,
    right_ts=landsat_ts,
    left_names=layer_names,
    right_names=layer_names,
    left_vis=landsat_vis,
    right_vis=landsat_vis,
)
Map.centerObject(roi, zoom=8)
Map

## Compare Landsat imagery and National Land Cover Database (NLCD)

More information about NLCD can be found at the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD).

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

In [ ]:
NLCD = ee.ImageCollection("USGS/NLCD")

In [ ]:
NLCD_layers = NLCD.aggregate_array("system:id").getInfo()
print(NLCD_layers)

In [ ]:
NLCD2001 = ee.Image("USGS/NLCD/NLCD2001").select("landcover")
NLCD2006 = ee.Image("USGS/NLCD/NLCD2006").select("landcover")
NLCD2011 = ee.Image("USGS/NLCD/NLCD2011").select("landcover")
NLCD2016 = ee.Image("USGS/NLCD/NLCD2016").select("landcover")

In [ ]:
NLCD = ee.ImageCollection([NLCD2001, NLCD2006, NLCD2011, NLCD2016])

In [ ]:
NLCD_layer_names = ["NLCD " + str(year) for year in range(2001, 2017, 5)]
print(NLCD_layer_names)

In [ ]:
roi = ee.Geometry.Polygon(
    [
        [
            [-115.897448, 35.640766],
            [-115.897448, 36.603608],
            [-113.784915, 36.603608],
            [-113.784915, 35.640766],
            [-115.897448, 35.640766],
        ]
    ],
    None,
    False,
)

In [ ]:
landsat_ts = geemap.landsat_timeseries(
    roi=roi, start_year=2001, end_year=2016, start_date="01-01", end_date="12-31"
)

In [ ]:
landsat_layer_names = ["Landsat " + str(year) for year in range(2001, 2017)]

In [ ]:
landsat_vis = {
    "min": 0,
    "max": 0.3,
    "gamma": [1, 1, 1],
    "bands": ["NIR", "Red", "Green"],
}

nlcd_vis = {"bands": ["landcover"]}

In [ ]:
Map = geemap.Map()
Map.ts_inspector(
    left_ts=landsat_ts,
    right_ts=NLCD,
    left_names=landsat_layer_names,
    right_names=NLCD_layer_names,
    left_vis=landsat_vis,
    right_vis=nlcd_vis,
)
Map.centerObject(roi, zoom=8)
Map